In [ ]:
%load_ext micropython_magic
%reload_ext micropython_magic

In [ ]:
%mpy -s {"/dev/cu.usbmodem1101"}

In [ ]:
# %%micropython

import time
from config import config
from machine import Pin, RTC
from mp_libs.network import Network
from mp_libs.protocols import espnow_protocol
from mp_libs.protocols import min_iot_protocol
from mp_libs.protocols.wifi_protocols import WifiProtocol
from mp_libs.time import ptp

TIMEOUT_MSEC = 5000
TARGET_TS = 795074941607459 + 360_000_000  # 11:09pm

rtc = RTC()
wifi = Network.create_wifi()
miniot_network = Network.create_min_iot("GTY")
epn = miniot_network.transport.transport._transport
wifi.connect()
miniot_network.connect()
wifi.ntp_time_sync()

print(f"Initial RTC datetime: {rtc.datetime()}")
print(f"Initial RTC now: {rtc.now()}")
print(f"My MAC: {epn.wifi._sta.config('mac')}")
print(f"EPN Peer: {config['epn_peer_mac']}")
print(f"Channel: {epn.wifi._sta.config('channel')}")
print(f"EPN timeout: {epn.epn.config('timeout_ms')}")

target = TARGET_TS
while True:
    rxed_packets = []
    data_available = False

    # Receive data
    while not data_available:
        data_available = miniot_network.receive(rxed_packets)
        time.sleep_ms(100)

    # Parse rx'ed packets
    for packet in rxed_packets:
        if ptp.is_ptp_msg(packet.msg):
            ptp_type, payload = ptp.parse_msg(packet.msg)
        else:
            print(f"Rx'ed unexpected message: {packet}")
            continue

        # Perform PTP sync
        if ptp_type == ptp.PtpMsg.SYNC_REQ:
            ptp.sequence_master(
                miniot_network.send,
                miniot_network.receive,
                lambda miniot_msg: miniot_msg.msg,
                TIMEOUT_MSEC,
                num_sync_cycles=payload)

            # print(f"now: {rtc.datetime()}")
            # print(f"RTC now: {rtc.now()}")

    target += 15_000_000
    while rtc.now() < target:
        pass

    print(f"{rtc.now()}")
